# Programming Exercise 2: Logistic Regression

## Introduction

In this exercise, you will implement logistic regression and apply it to two different datasets. 

In [2]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# library written for this exercise providing additional functions for assignment submission, and others
import utils

# tells matplotlib to embed plots within the notebook
%matplotlib inline

## 1 Logistic Regression

In this part of the exercise, you will build a logistic regression model to predict whether a student gets admitted into a university. Suppose that you are the administrator of a university department and
you want to determine each applicant’s chance of admission based on their results on two exams. You have historical data from previous applicants that you can use as a training set for logistic regression. For each training example, you have the applicant’s scores on two exams and the admissions
decision. Your task is to build a classification model that estimates an applicant’s probability of admission based the scores from those two exams. 

The following cell will load the data and corresponding labels:

In [190]:
# Load data
# The first two columns contains the exam scores and the third column
# contains the label.
data = np.loadtxt(os.path.join('Data', 'ex2data1.txt'), delimiter=',')

np.random.shuffle(data)

X, y = data[:, 0:2], data[:, 2]

# Setup the data matrix appropriately, and add ones for the intercept term

# Add intercept term to X
X = np.concatenate([np.ones((data.shape[0], 1)), X], axis=1)


In [174]:
new_X = np.zeros((X.shape[0],5))
new_X[:,0:3] = X[:,0:3]
new_X[:,3] = X[:,1]**2
new_X[:,4] = X[:,2]**3


# 60% training set 
X_train = new_X[0:60,:]
y_train = y[0:60]

# 20% cross validation set
X_cross = new_X[60:80,:]
y_cross = y[60:80]

# 20% testing set
X_test = new_X[80:100,:]
y_test = y[80:100]

In [131]:
X_train

array([[1.00000000e+00, 5.04581598e+01, 7.58098595e+01, 2.54602589e+03,
        4.35689482e+05],
       [1.00000000e+00, 8.22266616e+01, 4.27198785e+01, 6.76122387e+03,
        7.79632666e+04],
       [1.00000000e+00, 9.92725269e+01, 6.09990310e+01, 9.85503460e+03,
        2.26970183e+05],
       [1.00000000e+00, 3.58474088e+01, 7.29021980e+01, 1.28503672e+03,
        3.87455534e+05],
       [1.00000000e+00, 7.47892530e+01, 4.15734152e+01, 5.59343236e+03,
        7.18533646e+04],
       [1.00000000e+00, 8.89138964e+01, 6.98037889e+01, 7.90568098e+03,
        3.40123774e+05],
       [1.00000000e+00, 4.04575510e+01, 9.75351855e+01, 1.63681343e+03,
        9.27863183e+05],
       [1.00000000e+00, 5.39710521e+01, 8.92073501e+01, 2.91287447e+03,
        7.09907750e+05],
       [1.00000000e+00, 6.18302060e+01, 5.02561079e+01, 3.82297438e+03,
        1.26930665e+05],
       [1.00000000e+00, 6.40393204e+01, 7.80316880e+01, 4.10103456e+03,
        4.75130605e+05],
       [1.00000000e+00, 5.5340

In [132]:
def sigmoid(z):
    """
    Compute sigmoid function given the input z.
    
    Parameters
    ----------
    z : array_like
        The input to the sigmoid function. This can be a 1-D vector 
        or a 2-D matrix. 
    
    Returns
    -------
    g : array_like
        The computed sigmoid function. g has the same shape as z, since
        the sigmoid is computed element-wise on z.
        
    Instructions
    ------------
    Compute the sigmoid of each value of z (z can be a matrix, vector or scalar).
    """
    # convert input to a numpy array
    z = np.array(z)
    
    # You need to return the following variables correctly 
    g = np.zeros(z.shape)

    # ====================== YOUR CODE HERE ======================
    g = 1 / (1 + np.exp(-z))


    # =============================================================
    return g

In [133]:
def costFunction(theta, X, y):
    """
    Compute cost and gradient for logistic regression. 
    
    Parameters
    ----------
    theta : array_like
        The parameters for logistic regression. This a vector
        of shape (n+1, ).
    
    X : array_like
        The input dataset of shape (m x n+1) where m is the total number
        of data points and n is the number of features. We assume the 
        intercept has already been added to the input.
    
    y : arra_like
        Labels for the input. This is a vector of shape (m, ).
    
    Returns
    -------
    J : float
        The computed value for the cost function. 
    
    grad : array_like
        A vector of shape (n+1, ) which is the gradient of the cost
        function with respect to theta, at the current values of theta.
        
    Instructions
    ------------
    Compute the cost of a particular choice of theta. You should set J to 
    the cost. Compute the partial derivatives and set grad to the partial
    derivatives of the cost w.r.t. each parameter in theta.
    """
    # Initialize some useful values
    m = y.size  # number of training examples

    # You need to return the following variables correctly 
    J = 0
    grad = np.zeros(theta.shape)

    # ====================== YOUR CODE HERE ======================
    h = sigmoid(X.dot(theta.T))
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    grad = (1 / m) * (h - y).dot(X)
    
    
    # =============================================================
    return J, grad

In [134]:
def computeCost(theta,X,y):
    # Initialize some useful values
    m = y.size  # number of training examples

    # You need to return the following variables correctly 
    J = 0

    # ====================== YOUR CODE HERE ======================
    h = sigmoid(X.dot(theta.T))
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    
    return J

Once you are done call your `costFunction` using two test cases for  $\theta$ by executing the next cell.

In [189]:
# set options for optimize.minimize
options= {'maxiter': 400}

# see documention for scipy's optimize.minimize  for description about
# the different parameters
# The function returns an object `OptimizeResult`
# We use truncated Newton algorithm for optimization which is 
# equivalent to MATLAB's fminunc
# See https://stackoverflow.com/questions/18801002/fminunc-alternate-in-numpy



# Initialize fitting parameters
initial_theta = np.zeros(3)



res = optimize.minimize(costFunction,
                        initial_theta,
                        (X_train[:,0:3], y_train),
                        jac=True,
                        method='TNC',
                        options=options)

# the fun property of `OptimizeResult` object returns
# the value of costFunction at optimized theta
cost = res.fun

# the optimized theta is in the x property
theta = res.x

# Print theta to screen
print('Cost at theta found by optimize.minimize: {:.3f}'.format(cost))

print('theta:')
print('\t[{:.3f}, {:.3f}, {:.3f}]'.format(*theta))


Cost at theta found by optimize.minimize: 0.204
theta:
	[-23.135, 0.195, 0.186]


In [138]:
def predict(theta, X):
    """
    Predict whether the label is 0 or 1 using learned logistic regression.
    Computes the predictions for X using a threshold at 0.5 
    (i.e., if sigmoid(theta.T*x) >= 0.5, predict 1)
    
    Parameters
    ----------
    theta : array_like
        Parameters for logistic regression. A vecotor of shape (n+1, ).
    
    X : array_like
        The data to use for computing predictions. The rows is the number 
        of points to compute predictions, and columns is the number of
        features.

    Returns
    -------
    p : array_like
        Predictions and 0 or 1 for each row in X. 
    
    Instructions
    ------------
    Complete the following code to make predictions using your learned 
    logistic regression parameters.You should set p to a vector of 0's and 1's    
    """
    m = X.shape[0] # Number of training examples

    # You need to return the following variables correctly
    p = np.zeros(m)

    # ====================== YOUR CODE HERE ======================
    p = np.round(sigmoid(X.dot(theta.T)))

    
    # ============================================================
    return p

After you have completed the code in `predict`, we proceed to report the training accuracy of your classifier by computing the percentage of examples it got correct.

In [139]:

# Compute accuracy on our training set
p = predict(theta, X_test[:,0:4])
print('Train Accuracy: {:.2f} %'.format(np.mean(p == y_test) * 100))


Train Accuracy: 100.00 %


In [140]:
print(p[0:11])
print(y_test[0:11])

[1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1.]
[1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1.]


## 2 Regularized logistic regression

In this part of the exercise, you will implement regularized logistic regression to predict whether microchips from a fabrication plant passes quality assurance (QA). During QA, each microchip goes through various tests to ensure it is functioning correctly.
Suppose you are the product manager of the factory and you have the test results for some microchips on two different tests. From these two tests, you would like to determine whether the microchips should be accepted or rejected. To help you make the decision, you have a dataset of test results on past microchips, from which you can build a logistic regression model.

First, we load the data from a CSV file:

<a id="section5"></a>
### 2.3 Cost function and gradient

Now you will implement code to compute the cost function and gradient for regularized logistic regression. Complete the code for the function `costFunctionReg` below to return the cost and gradient.

Recall that the regularized cost function in logistic regression is

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^m \left[ -y^{(i)}\log \left( h_\theta \left(x^{(i)} \right) \right) - \left( 1 - y^{(i)} \right) \log \left( 1 - h_\theta \left( x^{(i)} \right) \right) \right] + \frac{\lambda}{2m} \sum_{j=1}^n \theta_j^2 $$

Note that you should not regularize the parameters $\theta_0$. The gradient of the cost function is a vector where the $j^{th}$ element is defined as follows:

$$ \frac{\partial J(\theta)}{\partial \theta_0} = \frac{1}{m} \sum_{i=1}^m \left( h_\theta \left(x^{(i)}\right) - y^{(i)} \right) x_j^{(i)} \qquad \text{for } j =0 $$

$$ \frac{\partial J(\theta)}{\partial \theta_j} = \left( \frac{1}{m} \sum_{i=1}^m \left( h_\theta \left(x^{(i)}\right) - y^{(i)} \right) x_j^{(i)} \right) + \frac{\lambda}{m}\theta_j \qquad \text{for } j \ge 1 $$
<a id="costFunctionReg"></a>

In [156]:
def costFunctionReg(theta, X, y, lambda_):
    """
    Compute cost and gradient for logistic regression with regularization.
    
    Parameters
    ----------
    theta : array_like
        Logistic regression parameters. A vector with shape (n, ). n is 
        the number of features including any intercept. If we have mapped
        our initial features into polynomial features, then n is the total 
        number of polynomial features. 
    
    X : array_like
        The data set with shape (m x n). m is the number of examples, and
        n is the number of features (after feature mapping).
    
    y : array_like
        The data labels. A vector with shape (m, ).
    
    lambda_ : float
        The regularization parameter. 
    
    Returns
    -------
    J : float
        The computed value for the regularized cost function. 
    
    grad : array_like
        A vector of shape (n, ) which is the gradient of the cost
        function with respect to theta, at the current values of theta.
    
    Instructions
    ------------
    Compute the cost `J` of a particular choice of theta.
    Compute the partial derivatives and set `grad` to the partial
    derivatives of the cost w.r.t. each parameter in theta.
    """
    # Initialize some useful values
    m = y.size  # number of training examples

    # You need to return the following variables correctly 
    J = 0
    grad = np.zeros(theta.shape)

    # ===================== YOUR CODE HERE ======================
    h = sigmoid(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X) 
    grad = grad + (lambda_ / m) * temp
    # =============================================================
    return J, grad

In [160]:
def computeCostReg(theta,X,y,lambda_):
    # Initialize some useful values
    m = y.size  # number of training examples

    # You need to return the following variables correctly 
    J = 0

    h = sigmoid(X.dot(theta.T))
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(theta))
    
    return J

In [187]:

# Set regularization parameter lambda to 1 (you should vary this)
lambda_ = [0,0.01,0.05,0.1,0.5,1,5,10]
# set options for optimize.minimize
options= {'maxiter': 100}

for i in range(len(lambda_)):
    
    # Initialize fitting parameters
    initial_theta = np.zeros(5)
    
    res = optimize.minimize(costFunctionReg,
                            initial_theta,
                            (X_train[:,0:5], y_train, lambda_[i]),
                            jac=True,
                            method='TNC',
                            options=options)
    print('cost is equal = ',computeCostReg(res.x,X_cross[:,0:5],y_cross,lambda_[i]))


# the optimized theta is in the x property of the result





cost is equal =  0.3097337573003858
cost is equal =  0.3109055290958805
cost is equal =  0.31341241778878737
cost is equal =  0.31657023747496843
cost is equal =  0.31329802257101497
cost is equal =  0.32443726581422067
cost is equal =  0.3283905505620721
cost is equal =  0.335014127467089
